***GENERATED CODE FOR snowflakessink PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class SnowFlakeConnector:

    def fetch(spark, config):
        return spark.read.format("net.snowflake.spark.snowflake") \
            .option("sfUrl", eval(config)['access_url']) \
            .option("sfUser", eval(config)['username']) \
            .option("sfPassword", eval(config)['password']) \
            .option("sfDatabase", eval(config)['database']) \
            .option("sfSchema", eval(config)['schema']) \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .load()

    def put(df, spark, config):
        df.write.mode(eval(config)['writemode'])\
            .format("net.snowflake.spark.snowflake") \
            .option("sfUrl", eval(config)['access_url']) \
            .option("sfUser", eval(config)['username']) \
            .option("sfPassword", eval(config)['password']) \
            .option("sfDatabase", eval(config)['database']) \
            .option("sfSchema", eval(config)['schema']) \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class SnowFlakeConnector:

    def fetch(spark, config):
        return spark.read.format("net.snowflake.spark.snowflake") \
            .option("sfUrl", eval(config)['access_url']) \
            .option("sfUser", eval(config)['username']) \
            .option("sfPassword", eval(config)['password']) \
            .option("sfDatabase", eval(config)['database']) \
            .option("sfSchema", eval(config)['schema']) \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .load()

    def put(df, spark, config):
        df.write.mode(eval(config)['writemode'])\
            .format("net.snowflake.spark.snowflake") \
            .option("sfUrl", eval(config)['access_url']) \
            .option("sfUser", eval(config)['username']) \
            .option("sfPassword", eval(config)['password']) \
            .option("sfDatabase", eval(config)['database']) \
            .option("sfSchema", eval(config)['schema']) \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .save()


***READING DATAFRAME***

In [ ]:
#%run snowflakessinkHooks.ipynb
try:
	#sourcePreExecutionHook()

	snowflake = SnowFlakeConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/6040bdf3ce0beb63e0ae0a01/part-00000-ad1308ba-396c-4c33-83da-0700b81a7ab0-c000.csv', 'access_url': 'https://wx63330.southeast-asia.azure.snowflakecomputing.com', 'username': 'numtra1122', 'password': 'numtra1122F', 'database': 'TESTDB', 'schema': 'PUBLIC', 'qtype': 'dbtable', 'query': 'BOSTON_TRAIN', 'dbtype': 'snowflake', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(snowflake)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run snowflakessinkHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(snowflake,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "4.58", "stddev": "11.57", "min": "0.02187", "max": "73.5341", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "10.13", "stddev": "20.15", "min": "0.0", "max": "95.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "10.45", "stddev": "6.47", "min": "1.25", "max": "27.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "0.06", "stddev": "0.24", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "0.55", "stddev": "0.12", "min": "0.392", "max": "0.871", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "6.24", "stddev": "0.66", "min": "4.88", "max": "8.725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "70.95", "stddev": "27.42", "min": "7.8", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "3.96", "stddev": "2.14", "min": "1.3861", "max": "9.2203", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "9.15", "stddev": "8.43", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "399.95", "stddev": "164.04", "min": "188.0", "max": "711.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "18.19", "stddev": "2.09", "min": "13.0", "max": "21.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "357.25", "stddev": "94.17", "min": "3.65", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "13.59", "stddev": "7.33", "min": "2.87", "max": "31.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "102", "mean": "21.35", "stddev": "8.85", "min": "5.0", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run snowflakessinkHooks.ipynb
try:
	#sinkPreExecutionHook()

	snowflake = SnowFlakeConnector.put(spark, "{'access_url': 'https://wx63330.southeast-asia.azure.snowflakecomputing.com', 'username': 'numtra1122', 'password': 'numtra1122F', 'database': 'TESTDB', 'schema': '', 'dbtype': 'snowflake', 'table_option': 'create_new_table', 'query': 'boastpredict11', 'new_table_name': 'boastpredict11', 'writemode': '', 'qtype': 'dbtable', 'is_header': 'Use Header Line'}")
	#sinkPostExecutionHook(snowflake)

except Exception as ex: 
	logging.error(ex)
